In [ ]:
import bisect
import math

#import cv2
#import h5py

import matplotlib.pyplot as plt
import numpy as np
#import tqdm
import os
import folium

os.environ['USE_PYGEOS'] = '0'
import geopandas as gpd


from shapely import Point

import jaxlie
import plotly.graph_objects as go  
import trajectory_utils
import numpy as np
#import pandas as pd
import pyproj

In [ ]:
# load permitted parking data
permitted_parking_gdf = gpd.read_file('/home/sf_afn/Downloads/univrses_data/permitted_parking.json')
permitted_parking_gdf.head()

In [ ]:
# load vehicle detections
car_triangulations_gdf = gpd.read_file("/home/sf_afn/Downloads/univrses_data/parked_car_triangulations_2023_04_24.csv")

In [ ]:
# --->BK1 base map declare
# create a map centered on Sodermalm
bbox_center = [59.312345, 18.066072] 
map_1 = folium.Map(location= bbox_center, zoom_start=14,tiles="cartodbpositron" )

In [ ]:
permitted_parking_gdf.crs

In [ ]:
permitted_parking_gdf  = permitted_parking_gdf.to_crs(epsg=4326)
permitted_parking_gdf['xy']= permitted_parking_gdf ['geometry'].apply(lambda row: row.xy )
permitted_parking_gdf['location_coords'] = permitted_parking_gdf ['xy'].apply(lambda row: [(x,y) for x,y in zip(row[1],row[0])])

In [ ]:
permitted_parking_gdf.head()

In [ ]:
permitted_parking_gdf['geometry'][100:1800].geom_type

In [ ]:
# ----> BK1 permitted parking polylines
# add geometry to a feature group
ls_fg = folium.FeatureGroup(name= "permitted-parking")
#polyline held as child of the map
for linestring in permitted_parking_gdf['location_coords']:
    ls_fg.add_child(folium.PolyLine(locations= linestring))
# feature group added to the child
# warning: add layer control after adding your last layer otherwise nothing will work as its supposed to!
map_1.add_child(ls_fg)


In [ ]:
car_triangulations_gdf.crs

In [ ]:
car_triangulations_gdf.head()

In [ ]:
car_triangulations_gdf['geometry'] = car_triangulations_gdf.apply(lambda row: Point(row['longitude'], row['latitude']), axis= 1)

In [ ]:
car_triangulations_gdf.set_crs(epsg=4326, inplace= True)

In [ ]:
car_triangulations_gdf.to_crs(epsg= 3011, inplace=True)

In [ ]:
car_triangulations_gdf.crs

In [ ]:
car_triangulations_gdf.explore()

In [ ]:
triangulation = car_triangulations_gdf['geometry'].iloc[42]

In [ ]:
triangulation_point= [triangulation.x, triangulation.y]
triangulation_point

In [ ]:
nearby_parking_ids = []
for idx, row in permitted_parking_gdf.iterrows():
    dist = triangulation_point.distance(row.geometry)
    if dist < 100:
        nearby_parking_ids.append(idx)


In [ ]:
sindex_car_tri = car_triangulations_gdf.sindex

In [ ]:
parking_sindex= permitted_parking_gdf.sindex

In [ ]:
parking_sindex

In [ ]:
tri_buff = triangulation.buffer(100)
close_parking_idx = list(parking_sindex.intersection(tri_buff).bounds)
